In [5]:

from keys import * 

import alpaca_trade_api as tradeapi



In [3]:

api = tradeapi.REST(API_KEY, SECRET_KEY, api_version='v2') # or use ENV Vars shown below
account = api.get_account()

portfolio = api.list_positions()

# Print the quantity of shares for each position.
for position in portfolio:
    print("{} shares of {}".format(position.qty, position.symbol))

# Check how much money we can use to open new positions.
print('${} is available as buying power.'.format(account.buying_power))


$400000 is available as buying power.


In [4]:


# Get a list of all active assets.
active_assets = api.list_assets(status='active')
print('all assets: ', len(active_assets))

# Filter the assets down to just those on NASDAQ.
nasdaq_assets = [a for a in active_assets if a.exchange == 'NASDAQ']
print('nasdaq_assets: ', len(nasdaq_assets))

# Check if AAPL is tradable on the Alpaca platform.
aapl_asset = api.get_asset('AAPL')
if aapl_asset.tradable:
    print('We can trade AAPL.')
    

all assets:  8640
nasdaq_assets:  3400
We can trade AAPL.


In [6]:

# Check if the market is open now.
clock = api.get_clock()
print('The market is {}'.format('open.' if clock.is_open else 'closed.'))

# Check when the market was open on Dec. 1, 2018
date = '2018-12-01'
calendar = api.get_calendar(start=date, end=date)[0]
print('The market opened at {} and closed at {} on {}.'.format(
    calendar.open,
    calendar.close,
    date
))


The market is closed.
The market opened at 09:30:00 and closed at 16:00:00 on 2018-12-01.


In [7]:

# Get daily price data for AAPL over the last 5 trading days.
barset = api.get_barset('AAPL', 'day', limit=5)
aapl_bars = barset['AAPL']
print(barset)

# See how much AAPL moved in that timeframe.
week_open = aapl_bars[0].o
week_close = aapl_bars[-1].c
percent_change = (week_close - week_open) / week_open * 100
print('AAPL moved {}% over the last 5 days'.format(percent_change))


{'AAPL': [Bar({   'c': 323.42,
    'h': 323.44,
    'l': 318.93,
    'o': 320.745,
    't': 1591070400,
    'v': 37968664}), Bar({   'c': 325.09,
    'h': 326.1,
    'l': 322.3,
    'o': 324.66,
    't': 1591156800,
    'v': 45506089}), Bar({   'c': 322.25,
    'h': 325.62,
    'l': 320.78,
    'o': 324.39,
    't': 1591243200,
    'v': 38930216}), Bar({   'c': 331.54,
    'h': 331.75,
    'l': 323.23,
    'o': 323.35,
    't': 1591329600,
    'v': 29572650}), Bar({   'c': 333.4402,
    'h': 333.5,
    'l': 327.32,
    'o': 330.25,
    't': 1591588800,
    'v': 20899464})]}
AAPL moved 3.958035199301626% over the last 5 days


In [9]:

# Submit a market order to buy 1 share of Apple at market price
api.submit_order(
    symbol='AAPL',
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc'
)

# Submit a limit order to attempt to sell 1 share of AMD at a
# particular price ($20.50) when the market opens
api.submit_order(
    symbol='AMD',
    qty=1,
    side='buy',
    type='limit',
    time_in_force='opg', # 'gtc', 'day'
    limit_price=20.50,
    client_order_id='my_first_order'
)



# Get our order using its Client Order ID.
my_order = api.get_order_by_client_order_id('my_first_order')
print('Got order #{}'.format(my_order.id))


Got order #4345eeaa-38a0-4bb4-ba9f-9b0d498f630f


In [11]:

symbol = 'AAPL'
symbol_bars = api.get_barset(symbol, 'minute', 1).df.iloc[0]
symbol_price = symbol_bars[symbol]['close']

# We could buy a position and add a stop-loss and a take-profit of 5 %
api.submit_order(
    symbol=symbol,
    qty=1,
    side='buy',
    type='market',
    time_in_force='gtc',
    order_class='bracket',
    stop_loss={'stop_price': symbol_price * 0.95,
               'limit_price':  symbol_price * 0.94},
    take_profit={'limit_price': symbol_price * 1.05}
)


Order({   'asset_class': 'us_equity',
    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
    'canceled_at': None,
    'client_order_id': '772a4ff2-013f-4f77-8ebe-5f28d709659d',
    'created_at': '2020-06-09T06:03:21.279519Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '588c0346-b53e-443d-b7c5-56e5a6f4f3dd',
    'legs': [   {   'asset_class': 'us_equity',
                    'asset_id': 'b0b6dd9d-8b9b-48a9-ba46-b9d54906e415',
                    'canceled_at': None,
                    'client_order_id': 'e31ee70f-c3e8-43eb-819a-3a3aaab9ab27',
                    'created_at': '2020-06-09T06:03:21.279519Z',
                    'expired_at': None,
                    'extended_hours': False,
                    'failed_at': None,
                    'filled_at': None,
                    'filled_avg_price': None,
                    'filled_qty': '0',
              

In [10]:
# Get the last 100 of our closed orders
closed_orders = api.list_orders(
    status='closed',
    limit=100,
    nested=True  # show nested multi-leg orders
)

# Get only the closed orders for a particular stock
closed_aapl_orders = [o for o in closed_orders if o.symbol == 'AAPL']
print(closed_aapl_orders)


[]
